# Set up

In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from datetime import date
import io
import random


import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
parent_path = 'C:\\Users\\DONGZ027\\Desktop\\Local Crafts\\craft001_Baymax'

mmmYear = 2023
ticket_price = 105

media_mapping = {
    'NTV':'National TV', 
    'LTV': 'Local TV',
    'CTV': 'Connected TV',
    'TTV' : 'TTV',
    'STR' : 'Streaming',
    'YOT' : 'YouTube',
    'CIN' : 'Cinema',
    'DIS' : 'Display',
    'SEM' : 'Paid Search',
    'FAB' : 'Facebook',
    'SNP' : 'Snapchat',
    'TWT' : 'Twitter',
    'PIN' : 'Pinterest',
    'TIK' : 'TikTok',
    'DRD' : 'Digital Radio',
    'LRD' : 'Local Radio',
    'OOH' : 'Out of Home'
}

media_mapping_reverse = {v: k for k, v in media_mapping.items()}

spend_prefix = "M_P_"  
inc_prefix = "TIncT_P_" 
cpt_prefix = "CPT_P_"
mcpt_prefix = "MCPT_P_" 
minc_prefix = "nMTIncT_P_"

months_abbv = ['Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']
months_full = ['October', 'November', 'December', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September'] 

In [3]:
# Time reference
file_path = f"{parent_path}\data\DT_snowflake.csv"
time_ref = pd.read_csv(file_path)

time_ref = time_ref[[
    'FIS_WK_END_DT', 'FIS_YR_NB', 'FIS_QTR_NB', 'FIS_MO_NB', 'FIS_WK_NB', 
]].drop_duplicates() 
time_ref = time_ref[time_ref['FIS_YR_NB'] >= mmmYear]
time_ref['FIS_WK_END_DT'] = pd.to_datetime(time_ref['FIS_WK_END_DT']).dt.date
time_ref = time_ref.sort_values(by=['FIS_WK_END_DT']) 

counting_months = time_ref.groupby(['FIS_YR_NB', 'FIS_MO_NB']).size().reset_index(name='weeks_count') 
counting_months['lag_weeks'] = counting_months.groupby(['FIS_YR_NB'])['weeks_count'].cumsum().shift(fill_value=0) 
counting_months.loc[counting_months.FIS_MO_NB == 1, 'lag_weeks'] = 0

time_ref = time_ref.merge(counting_months, how = 'left', on = ['FIS_YR_NB', 'FIS_MO_NB']).drop_duplicates() 

counting_months2 = time_ref[['FIS_YR_NB', 'FIS_MO_NB']].drop_duplicates()
counting_months2 = counting_months2.reset_index(drop=True)
counting_months2['lag_months'] = counting_months2.index   

time_ref = time_ref.merge(counting_months2, how = 'left', on = ['FIS_YR_NB', 'FIS_MO_NB']).drop_duplicates() 
df_time = time_ref.reset_index()

In [4]:
# Media timing file
# **********************************************************************************
file_path = f"{parent_path}\\data\\UK\\input_mediaTiming.csv"
df_curve  = pd.read_csv(file_path)
media_list = df_curve.columns



# Media parameters file
# **********************************************************************************
file_path = f"{parent_path}\\data\\UK\\input_300pct.csv"
df_params  = pd.read_csv(file_path)
names = list(df_params.columns)
names2 = [s.replace("FABING", "ING") for s in names]
names2 = [s.replace("DIS_BAN", "BAN") for s in names2]
names2 = [s.replace("DIS_AFF", "AFF") for s in names2]
names2 = [s.replace("TlncT", "TIncT") for s in names2]
df_params.columns = names2

In [5]:
def processing_spend_df(file_path):
    data = pd.read_csv(file_path)
    data.rename(columns=media_mapping_reverse, inplace=True)
    data = data.reset_index()
    data.rename(columns={"index": "FIS_MO_NB"}, inplace=True)
    data['FIS_MO_NB'] = data['FIS_MO_NB'] + 1
    data.drop(columns=['Month'], inplace=True)
    # data.replace({0 : 10}, inplace=True)
    return data


scenario1 = processing_spend_df(f"{parent_path}\\samples\DLP_UK_FY23.csv")
scenario2 = processing_spend_df(f"{parent_path}\\samples\DLP_UK_FY23_110.csv")
base_year = scenario2


In [6]:
df_bounds = pd.DataFrame({
    'Media' : media_list,
    'LB' : 0.8,
    'UB' : 1.2
})

# Mature Functions

In [7]:
def compute_reward_X(X, spend_data, planning_year, threshold):
    """
    Compute the rewards over a 104-week period for a given media type and calculate
    marginal increments and costs, with an updated multiplier calculation based on a threshold.

    Args:
        X (str): The media type (e.g., "X1").
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'X1', 'X2', 'X3').
        planning_year (int): The fiscal year of the current spending plan.
        threshold (float): A value between 0 and 100. Months with spending percentage
            greater than threshold/100 will be counted towards the multiplier.

    Returns:
        tuple: A tuple containing:
            - reward_df (pd.DataFrame): DataFrame of shape (104, 13) with the reward curves.
            - minc_X (float): Total marginal rewards for media X over 12 months.
            - mc_X (float): Total marginal costs for media X over 12 months.
    """
    # Ensure global access to necessary dataframes
    global df_curve, df_params, df_time

    # ================================================================================================================
    # Step 1: Extract the spending array and compute the multiplier
    # ================================================================================================================
    spend_array = spend_data[X].values  # Spending for media X over 12 months
    # Compute total spending S_total
    S_total = spend_array.sum()

    # Handle the case when total spending is zero to avoid division by zero
    if S_total == 0:
        spend_percentages = np.zeros_like(spend_array)
    else:
        # Compute the percentage of total spend per month
        spend_percentages = spend_array / S_total

    # Compute the multiplier based on the threshold
    multiplier = np.sum(spend_percentages > (threshold / 100))

    # If multiplier is zero, set it to 1 to avoid division by zero later
    if multiplier == 0:
        multiplier = 1

    # ================================================================================================================
    # Step 2: Compute the timing stuff
    # ================================================================================================================
    # Prepare a list to hold the 104-length arrays for each month
    monthly_arrays = []

    # Get fiscal years and months from spend_data
    fiscal_months = spend_data['FIS_MO_NB'].values

    # Filter df_time for the relevant fiscal year
    df_time_filtered = df_time[df_time['FIS_YR_NB'] == planning_year]

    # Group by fiscal month and count the number of weeks in each month
    weeks_in_month = df_time_filtered.groupby('FIS_MO_NB')['FIS_WK_NB'].nunique()
    weeks_in_month = weeks_in_month.reindex(range(1, 13), fill_value=0)

    # Compute cumulative weeks to determine the starting week for each month
    cumulative_weeks = weeks_in_month.cumsum()
    start_weeks = [0] + cumulative_weeks.values.tolist()[:-1]

    # Initialize total marginal increments and costs
    minc_X = 0.0
    mc_X = 0.0

    # ================================================================================================================
    # Loop over each month to compute the rewards
    # ================================================================================================================
    for idx, S_mo in enumerate(spend_array):
        fiscal_month = fiscal_months[idx]

        if S_mo == 0:
            # If the spending is zero, append an array of zeros
            monthly_array = np.zeros(104)
            monthly_arrays.append(monthly_array)
            continue

        # Step 2: Calculate annualized spending and find the closest match in df_params
        S_yr = S_mo * multiplier
        M_P_X_col = spend_prefix + X
        TIncT_P_X_col = inc_prefix + X
        nMTIncT_P_X_col = minc_prefix + X  # Marginal rewards column
        Pct_Delta_col = 'Pct_Delta'   

        # Find the index of the closest spending value in df_params
        idx_closest = (np.abs(df_params[M_P_X_col] - S_yr)).idxmin()

        # Retrieve values from df_params
        reward_yr = df_params.at[idx_closest, TIncT_P_X_col]
        reward_mo = reward_yr / multiplier

        # Calculate minc_mo and mc_mo for marginal cost per reward
        minc_mo = df_params.at[idx_closest, nMTIncT_P_X_col]
        mc_mo = df_params.at[idx_closest, Pct_Delta_col] * S_yr


        # Accumulate total marginal increments and costs
        minc_X += minc_mo
        mc_X += mc_mo

        # Step 3: Generate the reward curve for the month
        timing_curve = df_curve[X].values  # Timing curve of length 52
        monthly_reward_curve = reward_mo * timing_curve


        # Step 4: Create a 104-length array with appropriate leading zeros
        start_week = start_weeks[fiscal_month - 1]
        leading_zeros = int(start_week)
        trailing_zeros = 104 - leading_zeros - 52
        if trailing_zeros < 0:
            # If there is an overlap beyond 104 weeks, truncate the array
            monthly_array = np.concatenate([
                np.zeros(leading_zeros),
                monthly_reward_curve[:104 - leading_zeros]
            ])
        else:
            monthly_array = np.concatenate([
                np.zeros(leading_zeros),
                monthly_reward_curve,
                np.zeros(trailing_zeros)
            ])
        monthly_arrays.append(monthly_array)
    

    #Aggregate the monthly arrays into a final reward curve
    monthly_arrays = [arr[:104] for arr in monthly_arrays]  # Ensure each array is at most 104 elements
    # Pad any shorter arrays to exactly 104 elements
    monthly_arrays = [np.pad(arr, (0, 104 - len(arr)), 'constant') if len(arr) < 104 else arr for arr in monthly_arrays]
    reward_X = np.sum(monthly_arrays, axis=0)


    # Construct the output DataFrame
    columns = [f'Month_{i+1}' for i in range(12)] + ['aggregated']
    data = np.column_stack(monthly_arrays + [reward_X])
    reward_df = pd.DataFrame(data, columns=columns)

    return reward_df, minc_X, mc_X


In [8]:
# check = compute_reward_X('NTV', scenario1, 2024, 0.9)

In [9]:
def compute_plan_reward(spend_data, planning_year, lead_years, lag_years, threshold):
    """
    Compute the reward curves for all medias over an extended time frame, broken down by month and aggregated.

    Args:
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'X1', 'X2', 'X3').
        lead_years (int): Number of years to add as leading zeros.
        lag_years (int): Number of years to add as trailing zeros.2
        planning_year (int): The fiscal year of the current spending plan.
        threshold (float): Threshold value for reward calculations.

    Returns:
        list: A list of 13 numpy arrays:
            - First 12 arrays represent the monthly rewards summed across all media
            - Last array represents the aggregated rewards summed across all media
            Each array includes leading and trailing zeros.
    """
    # Ensure global access to necessary dataframes
    # global df_curve, df_params, df_time

    # Step 1: Get the list of media columns from spend_data
    medias = spend_data.columns.tolist()[1:]

    # Initialize lists to store the monthly and aggregated reward arrays
    monthly_rewards = [[] for _ in range(12)]  # One list for each month
    aggregated_rewards = []  # List for aggregated rewards

    # Loop over each media to compute its reward curves
    for media in medias:
        # Extract spending data for the media
        media_spend_data = spend_data[['FIS_MO_NB', media]]

        # Call compute_reward_X for the current media
        reward_df = compute_reward_X(media, media_spend_data, planning_year, threshold)[0]

        # Extract monthly columns and aggregated column
        for month in range(12):
            month_col = f'Month_{month+1}'
            monthly_rewards[month].append(reward_df[month_col].values)
        
        # Extract and store aggregated rewards
        aggregated_rewards.append(reward_df['aggregated'].values)

    # Step 2: Sum the reward arrays across all media for each month and aggregated
    summed_monthly_rewards = [np.sum(month_arrays, axis=0) for month_arrays in monthly_rewards]
    summed_aggregated_rewards = np.sum(aggregated_rewards, axis=0)

    # Step 3: Add leading and trailing zeros to each array
    leading_zeros = np.zeros(lead_years * 52)
    trailing_zeros = np.zeros(lag_years * 52)

    # Create final list of extended arrays (12 monthly + 1 aggregated)
    extended_reward_curves = []
    
    # Process monthly arrays
    for monthly_reward in summed_monthly_rewards:
        extended_monthly = np.concatenate([leading_zeros, monthly_reward, trailing_zeros])
        extended_reward_curves.append(extended_monthly)
    
    # Process aggregated array
    extended_aggregated = np.concatenate([leading_zeros, summed_aggregated_rewards, trailing_zeros])
    extended_reward_curves.append(extended_aggregated)

    return extended_reward_curves


In [10]:
# check = compute_plan_reward(scenario1, 2024, 1, 1, 0.9)

In [11]:
def plan_forecast_craft(spend_data, planning_year, lead_years, lag_years, cutoff):
    """
    Generate monthly and quarterly forecast tables based on spending data and reward calculations.

    Args:
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'NTV', 'ING', 'STR').
        planning_year (int): The fiscal year of the current spending plan.
        lead_years (int): Number of years to add as leading zeros.
        lag_years (int): Number of years to add as trailing zeros.
        cutoff (float): Threshold value for reward calculations.

    Returns:
        tuple: A tuple containing:
            - craft_mo (pd.DataFrame): DataFrame with monthly forecast data.
            - craft_qtr (pd.DataFrame): DataFrame with quarterly forecast data.
    """
    df_time_scenario = df_time[df_time['FIS_YR_NB'].between(planning_year - lead_years, planning_year + lag_years)]
    weekly_table = df_time_scenario[['FIS_WK_END_DT', 'FIS_YR_NB', 'FIS_QTR_NB', 'FIS_MO_NB']]
    results = compute_plan_reward(spend_data, planning_year, lead_years, lag_years, cutoff)

    names = []
    for i in range(len(results)-1):
        serl = list(results[i])
        if len(serl) < weekly_table.shape[0]:
            serl = serl + [0] * (weekly_table.shape[0] - len(serl))
        if len(serl) > weekly_table.shape[0]:
            serl = serl[:weekly_table.shape[0]]
        col_name = str(planning_year) + ' ' + months_abbv[i]
        names.append(col_name)
        weekly_table[col_name] = serl

    # Monthly Table
    # ================================================================================================================
    monthly_table = weekly_table.groupby(['FIS_YR_NB', 'FIS_MO_NB'])[names].sum().reset_index()
    rewards = monthly_table.iloc[:, 2:].values.T
    monthly_table.FIS_MO_NB.replace(dict(zip(range(1, 13), months_abbv)), inplace=True) 
    monthly_table['timeline'] = monthly_table.FIS_YR_NB.astype(str) + " " + monthly_table.FIS_MO_NB.astype(str) 
    
    shard1 = pd.DataFrame({'Spending Month': names, "Spend": spend_data.iloc[:, 1:].sum(axis = 1).values})
    shard2 = pd.DataFrame(rewards)
    shard2.columns = monthly_table['timeline'].values
    craft_mo = pd.concat([shard1, shard2], axis=1) 


    # Monthly Table
    # ================================================================================================================
    quarter_table = weekly_table.groupby(['FIS_YR_NB', 'FIS_QTR_NB'])[names].sum().reset_index()
    rewards = quarter_table.iloc[:, 2:].values.T 
    rewards = rewards.reshape(4, 3, -1) # Turning monthly tracking into quarterly tracking
    rewards = rewards.sum(axis = 1)
    quarter_table.FIS_QTR_NB = quarter_table.FIS_QTR_NB.astype(str)
    quarter_table['timeline'] = quarter_table.FIS_YR_NB.astype(str) + " Q" + quarter_table.FIS_QTR_NB.astype(str)

    names = [str(planning_year) + " Q" + str(x) for x in range(1, 5)]
    shard1 = pd.DataFrame({'Spending Quarter': names, 
                           "Spend": spend_data.iloc[:, 1:].values.sum(axis = 1).reshape(4, 3).sum(axis = 1)})
    shard2 = pd.DataFrame(rewards)
    shard2.columns = quarter_table['timeline'].values
    craft_qtr = pd.concat([shard1, shard2], axis=1) 

    return craft_mo, craft_qtr
    

In [12]:
# Fore creating the final forecast table
# **********************************************************************************

shard_base = plan_forecast_craft(base_year, 2023, 0, 2, 0.9)[0]
shard_plan = plan_forecast_craft(scenario1, 2024, 1, 1, 0.9)[0]
check = pd.concat([shard_base, shard_plan], axis = 0)

shard1 = ['Total Attendance', ""]
shard2 = list(check.iloc[:, 2:].sum(axis = 0).values)

check.loc[-1] = np.array(shard1 + shard2)
check = check.reset_index(drop = True)

shard1 = check.iloc[:-1]
shard2 = check.iloc[[-1]]

check = pd.concat([shard2, shard1], axis = 0).reset_index(drop = True)
check


,Spending Month,Spend,2023 Oct,2023 Nov,2023 Dec,2023 Jan,2023 Feb,2023 Mar,2023 Apr,2023 May,...,2025 Dec,2025 Jan,2025 Feb,2025 Mar,2025 Apr,2025 May,2025 Jun,2025 Jul,2025 Aug,2025 Sep
0,Total Attendance,,13124.536118922046,59879.37982521834,58707.53736567874,54774.255698560926,65160.23902757405,90546.45698865171,77365.46503561716,81344.04525897413,...,21196.146996018648,9892.974089323023,5033.3188847087495,3194.2688802780494,1364.0807474775331,555.8622985544666,0.0,0.0,0.0,0.0
1,2023 Oct,2089984.14474,13124.536119,51630.413817,36821.06422,17345.994823,8705.709787,5774.9007,2466.082659,1420.428866,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023 Nov,267913.96391,0.0,8248.966008,17833.325465,15195.943354,7363.649539,4359.44833,1825.302251,1051.316535,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023 Dec,196786.4644,0.0,0.0,4053.147681,12285.437324,12350.028926,2571.354014,76.886591,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023 Jan,953244.3685,0.0,0.0,0.0,9946.880197,30542.512201,34359.89314,3525.936245,169.691765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2023 Feb,1207504.3417,0.0,0.0,0.0,0.0,6198.338574,23166.157241,20166.650603,14986.730622,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2023 Mar,2099646.77947,0.0,0.0,0.0,0.0,0.0,20314.703564,44666.20632,37341.311648,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2023 Apr,255465.886157,0.0,0.0,0.0,0.0,0.0,0.0,4638.400366,14110.248128,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2023 May,1726203.25185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12264.317695,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2023 Jun,612396.14521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def build_plan_summary(spend_data, planning_year, threshold, unit_revenue):
    """
    Build a summary report of the spending plan with total spend, total reward,
    cost per reward, and marginal cost per reward for each media and aggregated.

    Args:
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'YOT', 'FAB').
        planning_year (int): The fiscal year of the current spending plan.

    Returns:
        pd.DataFrame: A DataFrame 'plan_summary' with columns:
            - 'Total Spend'
            - 'Total Reward'
            - 'Cost per Reward'
            - 'Marginal Cost per Reward'
        and rows:
            - 'Total' (aggregated over all medias)
            - One row per media (e.g., 'YOT', 'FAB')
    """
    # Ensure nonlocal access to necessary dataframes
    # nonlocal df_curve, df_params, df_time

    # Initialize dictionaries to store summary data
    total_spend_dict = {}
    total_reward_dict = {}
    minc_X_dict = {}
    mc_X_dict = {}

    # List of media columns (exclude 'FIS_MO_NB')
    medias = spend_data.columns.tolist()[1:]

    # Loop over each media to compute summary metrics
    for media in medias:
        # Extract spending data for the media
        media_spend_data = spend_data[['FIS_MO_NB', media]]


        # Total spend for media: sum of monthly spend
        total_spend = media_spend_data[media].sum()
        total_spend_dict[media] = total_spend


        # Call compute_reward_X for the current media
        reward_df, minc_X, mc_X = compute_reward_X(media, media_spend_data, planning_year, threshold)

        # Total reward for media: sum of the 'aggregated' column
        total_reward = reward_df['aggregated'].sum()
        total_reward_dict[media] = total_reward

        # Store minc_X and mc_X
        minc_X_dict[media] = minc_X
        mc_X_dict[media] = mc_X


    # Calculate aggregated totals
    total_spend_agg = sum(total_spend_dict.values())
    total_reward_agg = sum(total_reward_dict.values())
    minc_X_agg = sum(minc_X_dict.values())
    mc_X_agg = sum(mc_X_dict.values())


    # Build the summary DataFrame
    plan_summary = pd.DataFrame(columns=['Total Spend', 'Total Attendance', 'Cost per Attendance', 'Marginal Cost per Attendance'])

    # Add aggregated totals to the summary
    plan_summary.loc['Total'] = {
        'Total Spend': total_spend_agg,
        'Total Attendance': total_reward_agg,
        'Cost per Attendance': total_spend_agg / total_reward_agg if total_reward_agg != 0 else np.nan,
        'Marginal Cost per Attendance': mc_X_agg / minc_X_agg if minc_X_agg != 0 else np.nan
    }

    # Add individual media data to the summary
    for media in medias:
        total_spend = total_spend_dict[media]
        total_reward = total_reward_dict[media]
        minc_X = minc_X_dict[media]
        mc_X = mc_X_dict[media]

        plan_summary.loc[media] = {
            'Total Spend': total_spend,
            'Total Attendance': total_reward,
            'Cost per Attendance': total_spend / total_reward if total_reward != 0 else np.nan,
            'Marginal Cost per Attendance': mc_X / minc_X if minc_X != 0 else np.nan
        }

    # Reorder rows: aggregated totals first, then individual medias
    plan_summary = plan_summary.reset_index().rename(columns={'index': 'Media'})
    plan_summary = plan_summary[['Media', 'Total Spend', 'Total Attendance', 'Cost per Attendance', 'Marginal Cost per Attendance']]

    # Add ROAS and MROAS
    plan_summary['ROAS'] = unit_revenue * plan_summary['Total Attendance'] / plan_summary['Total Spend']
    plan_summary['MROAS'] = 1 + ((unit_revenue - plan_summary['Marginal Cost per Attendance']) / plan_summary['Marginal Cost per Attendance'])


    # Round up columns
    plan_summary['Total Spend'] = plan_summary['Total Spend'].astype(int)
    plan_summary['Total Attendance'] = plan_summary['Total Attendance'].astype(int)
    plan_summary['Cost per Attendance'] = plan_summary['Cost per Attendance'].round(1)
    plan_summary['Marginal Cost per Attendance'] = plan_summary['Marginal Cost per Attendance'].round(1)
    plan_summary['ROAS'] = plan_summary['ROAS'].round(1)
    plan_summary['MROAS'] = plan_summary['MROAS'].round(1)


    return plan_summary

In [14]:
build_plan_summary(base_year, 2024, 0.9, ticket_price) 

,Media,Total Spend,Total Attendance,Cost per Attendance,Marginal Cost per Attendance,ROAS,MROAS
0,Total,13630027,1028386,13.3,30.5,7.9,3.4
1,NTV,5678942,331380,17.1,62.7,6.1,1.7
2,STR,2205421,149220,14.8,34.3,7.1,3.1
3,BAN,410196,52691,7.8,16.3,13.5,6.4
4,RAD,265052,16515,16.0,29.7,6.5,3.5
5,AFF,77974,12020,6.5,11.0,16.2,9.6
6,CEM,14105,2048,6.9,9.2,15.2,11.4
7,SEM,1075893,118347,9.1,19.7,11.5,5.3
8,Total_Social,3902440,346163,11.3,26.9,9.3,3.9


# Scenrio Plots

In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [16]:
l1 = [1, 2, 3]
l2 = [2, 1, 2]
[max(l1[i], l2[i]) for i in np.arange(len(l1))]

[2, 2, 3]

In [17]:
np.arange(3)

array([0, 1, 2])

In [ ]:
def scenario_plots(scenarios, metrics, channels, colors, title, ylabel1, ylabel2, currency_symbol):
    # **********************************************************************************
    # Read numbers
    # **********************************************************************************
    board1 = scenarios[0]
    board2 = scenarios[1]

    metric1 = metrics[0]
    metric2 = metrics[1]

    metric1_s1 = board1[metric1].values
    metric1_s2 = board2[metric1].values
    max_metric1 =  max(max(metric1_s1), max(metric1_s2))

    metric2_s1 = board1[metric2].values
    metric2_s2 = board2[metric2].values
    max_metric2 = max(max(metric2_s1), max(metric2_s2))

    metric1_compare = [(s2 - s1) / s1 * 100 for s1, s2 in zip(metric1_s1, metric1_s2)]
    metric2_compare = [(s2 - s1) / s1 * 100 for s1, s2 in zip(metric2_s1, metric2_s2)]

    color1_s1 = colors[0]
    color1_s2 = colors[1]
    color2_s1 = colors[2]
    color2_s2 = colors[3]

    default_textpos = 0.5 * max_metric1

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # **********************************************************************************
    # The bar plots
    # **********************************************************************************
    # Scenario 1
    # ....................................................
    fig.add_trace(
        go.Bar(
            x= channels,
            y= metric1_s1,
            name= "1",
            marker_color= color1_s1
        ),
        secondary_y=False,
    )

    # Scenario 2
    # ....................................................
    fig.add_trace(
        go.Bar(
            x= channels,
            y= metric1_s2,
            name= "2",
            marker_color= color1_s2
        ),
        secondary_y=False,
    )




    # **********************************************************************************
    # Line plots
    # **********************************************************************************
    # Scenario 1
    # ....................................................
    fig.add_trace(
        go.Scatter(
            x=channels,
            y= metric2_s1,
            name="MROAS 1",
            line=dict(
                color= color2_s1, 
                width=3, dash='dash',
                shape='spline',  # This creates a smooth curve
                smoothing=1.3    # Adjust smoothing intensity (0.5-1.5 range works well)
                ),
            marker=dict(
                size=10,         # Larger marker size
                color=color2_s1,
                line=dict(
                    width=1,
                    color='white'
                )
            )

        ),
        secondary_y=True,
    )

    # Scenario 2
    # ....................................................
    fig.add_trace(
        go.Scatter(
            x=channels,
            y= metric2_s2,
            name="MROAS 2",
            line=dict(color= color2_s2, 
                      width=3, dash='dash',
                      shape='spline',  # This creates a smooth curve
                      smoothing=1.3    # Adjust smoothing intensity (0.5-1.5 range works well)
                ),
            marker=dict(
                size=10,         # Larger marker size
                color=color2_s2,
                line=dict(
                    width=1,
                    color='white'
                )
            )
        ),
        secondary_y=True,
    )




    # **********************************************************************************
    # Costmetics
    # **********************************************************************************

    # Annotations for metric 1 change
    for i, channel in enumerate(channels):
        change = metric1_compare[i]
        
        # Determine color based on change
        color = "green" if change >= 0 else "red"
        
        # Format text with plus/minus sign and percentage
        if change >= 0:
            text = f"+{change:.1f}%"  # Add plus sign for positive changes
        else:
            text = f"{change:.1f}%"   # Negative sign is automatically included
        
        # Improved positioning logic
        current_value = max(metric1_s1[i], metric1_s2[i])
        
        # If the value is very small (less than 5% of the maximum), use a fixed position
        if current_value < 0.03 * max_metric1:
            ypos = 0.12 * max_metric1  # Position at 15% of max height for very small values
        # If it's the maximum value, add a bit more space
        elif current_value >= 0.95 * max_metric1:
            ypos = 1.05 * max_metric1  # Position at 110% of max for the largest values
        # For medium values, position proportionally
        else:
            ypos = current_value + (0.125 * max_metric1)  # Position above the bar with consistent spacing
        
        # Add the annotation without arrows
        fig.add_annotation(
            x=channel,
            y=ypos, 
            text=text,
            showarrow=False,  # No arrow
            font=dict(
                color=color, 
                size=14,      # Slightly larger font for better visibility
                weight='bold' # Make it bold for emphasis
            ),
            align='center',
            bgcolor='rgba(255,255,255,0.7)',  # Semi-transparent white background
            bordercolor=color,
            borderwidth=1,
            borderpad=3
        )




    fig.update_layout(
        # Wider plot for spacing
        width=1300,
        height=700,
        # Extra large left margin
        margin=dict(t=80, r=50, b=100, l=150),
        # Title styling
        title=dict(
            text= title,
            font=dict(
                size=28,
                color= color1_s2,
                weight='bold'
            ),
            x=0.5
        ),
        # Other layout
        barmode='group',
        legend=dict(
            orientation="h",
            yanchor="bottom", 
            y=-0.15,
            xanchor="center",
            x=0.5
        ),
        paper_bgcolor='white',
        plot_bgcolor='white'
    )

    # Set x-axis properties
    fig.update_xaxes(
        title_text="",
        showgrid=False,
        showline=True,
        linewidth=2,
        linecolor='lightgray'
    )

    # Set y-axes properties
    fig.update_yaxes(
        title_text= ylabel1,
        title_font=dict(size=16),
        range=[0, 1.2 * max(max(metric1_s2), max(metric1_s1))],
        showgrid=True,
        gridcolor='lightgray',
        secondary_y=False,
        tickformat=','
    )

    fig.update_yaxes(
        title_text= ylabel2,
        title_font=dict(size=16),
        range=[0, 1.2 * max(max(metric2_s2), max(metric2_s1))],
        showgrid=False,
        secondary_y=True,
        tickprefix= currency_symbol,
        ticksuffix='.0'
    )

    # Show the figure
    fig.show()

In [28]:
scenario_plots(
    scenarios = [build_plan_summary(scenario2, 2024, 0.9, ticket_price).iloc[1:, :], 
                 build_plan_summary(scenario1, 2024, 0.9, ticket_price).iloc[1:, :]],

    metrics = ['Total Spend', 'MROAS'],

    channels = media_list,

    colors = ['rgb(174, 139, 113)', 
            'rgb(140, 63, 12)',
            'rgb(174, 139, 113)',
            'rgb(140, 63, 12)'
            
            ],

    title = "Media budget & MROAS variation per touchpoint", 

    ylabel1 = "", ylabel2= "", currency_symbol = "£" 
)

In [29]:
scenario_plots(
    scenarios = [build_plan_summary(scenario1, 2024, 0.9, ticket_price).iloc[1:, :], 
                 build_plan_summary(scenario2, 2024, 0.9, ticket_price).iloc[1:, :]],

    metrics = ['Total Attendance', 'Cost per Attendance'],

    channels = media_list,

    colors = ['rgb(188, 214, 150)', 
            'rgb(36, 84, 40)',
            'rgb(188, 214, 150)', 
            'rgb(36, 84, 40)'
            ],

    title = "Incremental attendance & CPA evolution", 

    ylabel1 = "", ylabel2= "", currency_symbol = "£" 
)